<a href="https://colab.research.google.com/github/hellomizuki/111/blob/main/%E2%80%9CNovelAI%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

使用官方前端 + 优化版后端，可突破75限制，支持所有模型


In [1]:
#@title ### 检查 GPU 工作状态

!nvidia-smi

Mon Jan  9 14:06:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@title ### 下载 Novel AI API 后端、模型 
#@markdown 如果下载速度太慢可尝试重新启动

%cd /content/
!apt install -y -qq aria2
!aria2c --summary-interval=5 -x 3 --allow-overwrite=true -Z \
  https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/naifu.tar \
  https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-latest.tar 
  
!echo "Decompressing..."
!tar xf naifu.tar && rm naifu.tar
!echo "Done."

 *** Download Progress Summary as of Thu Jan  5 12:18:03 2023 *** 
=
[#0c2049 3.7GiB/7.4GiB(50%) CN:3 DL:5.1MiB ETA:12m19s]
FILE: /content/naifu.tar
-

 *** Download Progress Summary as of Thu Jan  5 12:18:09 2023 *** 
=
[#0c2049 3.7GiB/7.4GiB(50%) CN:3 DL:5.5MiB ETA:11m18s]
FILE: /content/naifu.tar
-


01/05 12:18:10 [NOTICE] Shutdown sequence commencing... Press Ctrl-C again for emergency shutdown.

01/05 12:18:10 [NOTICE] Download GID#0c2049a9e688f8df not complete: /content/naifu.tar

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
757827|OK  |    12MiB/s|/content/animefull-latest.tar
0c2049|INPR|   5.6MiB/s|/content/naifu.tar

Status Legend:
(OK):download completed.(INPR):download in-progress.

aria2 will resume download if the transfer is restarted.
If there are any errors, then see the log file. See '-l' option in help/man page for details.
Decompressing...
Done.


In [ ]:
#@title ### 安装依赖
#@markdown 耐心等待安装完成

%cd /content/naifu
!pip install virtualenv && bash ./setup.sh
!curl -Ls https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz | tar zx -C /usr/bin
!curl -Lo /usr/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /usr/bin/cloudflared
!/content/naifu/venv/bin/python -m pip install -qq pytorch_lightning==1.7.7

/content/naifu
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 KB 44.0 MB/s eta 0:00:00
bash: ./setup.sh: No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 33.6M  100 33.6M    0     0  54.6M      0 --:--:-- --:--:-- --:--:--  148M
/bin/bash: /content/naifu/venv/bin/python: No such file or directory


下载中文汉化包并解压（可选）

In [ ]:
!curl -o /content/naifu/Naifu.zip https://api.suwanya.cn/api/AI/Naifu.zip
!unzip -o /content/naifu/Naifu.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4096k  100 4096k    0     0   164k      0  0:00:24  0:00:24 --:--:--  202k
Archive:  /content/naifu/Naifu.zip
  inflating: static/404.html         
  inflating: static/icons/novelai-round.png  
  inflating: static/index.html       
  inflating: static/manifest.json    
   creating: static/old/
  inflating: static/old/404.html     
   creating: static/old/icons/
  inflating: static/old/icons/novelai-round.png  
  inflating: static/old/index.html   
  inflating: static/old/manifest.json  
   creating: static/old/tokenizer/
  inflating: static/old/tokenizer/clip_tokenizer.json  
 extracting: static/old/version      
   creating: static/old/_next/
   creating: static/old/_next/MG2vPPHR3En34h0ZDGvU7/
   creating: static/old/_next/static/
   creating: static/old/_next/static/chunks/
  inflating: static/old/_next/static/chunks/140.a9

In [6]:
#@title ### 启动模型
#@markdown 访问输出的映射地址（以 `trycloudflare.com` / `bore.pub` 结尾）即可。
#@markdown - 请等待模型加载完成（出现`Application startup complete`字样）再访问
#@markdown - cloudflare 提供的服务偶尔会出现请求超时，可换用 bore 隧道

%cd /content/naifu
!sed -i 's/# export SAVE_FILES="1"/export SAVE_FILES="1"/g' run.sh
!bash run.sh & cloudflared tunnel --url localhost:6969 

[Errno 2] No such file or directory: '/content/naifu'
/content
sed: can't read run.sh: No such file or directory
/bin/bash: cloudflared: command not found
bash: run.sh: No such file or directory


In [4]:
#@title ### （可选）使用 7G 的 `animefull-latest` 模型运行
#@markdown 默认使用的是 4G 大小的 animefull-final-pruned 模型。如果想使用 7G 的 animefull-latest 模型，运行这个

%cd /content/
!tar xf animefull-latest.tar -C /content/naifu/models && rm animefull-latest.tar
!sed -i 's/map_location="cpu"/map_location="cuda"/g' /content/naifu/hydra_node/models.py

%cd /content/naifu
%env DTYPE=float16
%env CLIP_CONTEXTS=3
%env AMP=1
%env MODEL=stable-diffusion
%env DEV=True
%env MODEL_PATH=models/animefull-latest
%env ENABLE_EMA=1
%env VAE_PATH=models/animevae.pt
%env PENULTIMATE=1
%env PYTHONDONTWRITEBYTECODE=1
%env SAVE_FILES=1

!./venv/bin/python -m uvicorn --host 0.0.0.0 --port=6969 main:app & bore local 6969 --to bore.pub & cloudflared tunnel --url localhost:6969 

/content
tar: animefull-latest.tar: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
sed: can't read /content/naifu/hydra_node/models.py: No such file or directory
[Errno 2] No such file or directory: '/content/naifu'
/content
env: DTYPE=float16
env: CLIP_CONTEXTS=3
env: AMP=1
env: MODEL=stable-diffusion
env: DEV=True
env: MODEL_PATH=models/animefull-latest
env: ENABLE_EMA=1
env: VAE_PATH=models/animevae.pt
env: PENULTIMATE=1
env: PYTHONDONTWRITEBYTECODE=1
env: SAVE_FILES=1
/bin/bash: cloudflared: command not found
/bin/bash: bore: command not found
/bin/bash: ./venv/bin/python: No such file or directory
